In [2]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [3]:
pip install datasets

In [4]:
pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [5]:
!pip install accelerate -U

In [6]:
from transformers import  AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset, DatasetDict
import numpy as np
import pandas as pd
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, AutoPeftModelForSequenceClassification

# Create model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Create tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
# Load the IMDb dataset
dataset = load_dataset('imdb')

# Split the original training data into new training data and validation data
train_test_split = dataset['train'].train_test_split(test_size=0.1)

dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test'],
    'test': dataset['test']
})

print(dataset)

# Tokenize the whole dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
tokenized_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))
tokenized_validation_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(100))

print(tokenized_train_dataset)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 22500
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 100
})


In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
    }


# Training and evaluation
training_args = TrainingArguments(
    output_dir="./data/training",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],  # Use 'test' split as validation
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("./data/model")


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.218800,0.290366,0.924640,0.925333,0.916902,0.933920


In [14]:
# Create LoRA PEFT Model

pretrained_model = AutoModelForSequenceClassification.from_pretrained("./data/model")

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=1,
    lora_alpha=1,
    lora_dropout=0.1,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
    bias="none"
)

peft_model = get_peft_model(pretrained_model, peft_config)
peft_model.print_trainable_parameters()


peft_training_args = TrainingArguments(
    output_dir="./data/peft_training",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],  # Use 'test' split as validation
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

peft_trainer.train()
peft_trainer.save_model("./data/peft_model")

# Alternative PEFT configuration
peft_config2 = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
    bias="all"
)


peft_model2 = get_peft_model(pretrained_model, peft_config2)

peft_training_args2 = TrainingArguments(
    output_dir="./data/peft_training2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.5,
    load_best_model_at_end=True
)

peft_trainer2 = Trainer(
    model=peft_model2,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],  # Use 'test' split as validation
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

peft_trainer2.train()
peft_trainer2.save_model("./data/peft_model2")


trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9306847223789819


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.152700,0.241800,0.925480,0.925910,0.920601,0.931280


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./data/model - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./data/model - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.160200,0.256840,0.925680,0.926100,0.920899,0.931360


Checkpoint destination directory ./data/peft_training/checkpoint-2813 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./data/model - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./data/model - will assume that the vocabulary was not modified.
  warnings.warn(


In [23]:
from peft import AutoPeftModelForSequenceClassification
from evaluate import evaluator

pretrained_model = AutoModelForSequenceClassification.from_pretrained("./data/model")
pretrained_tokenizer = AutoTokenizer.from_pretrained("./data/model")

# Doesn't work: AttributeError: 'PeftModelForSequenceClassification' object has no attribute 'task'
# inference_peft_model = AutoPeftModelForSequenceClassification.from_pretrained("./data/peft_model")


inference_peft_model = AutoModelForSequenceClassification.from_pretrained("./data/peft_model")
inference_peft_tokenizer = AutoTokenizer.from_pretrained("./data/peft_model")

# Doesn't work: AttributeError: 'PeftModelForSequenceClassification' object has no attribute 'task'
# inference_peft_model2 = AutoPeftModelForSequenceClassification.from_pretrained("./data/peft_model2")

inference_peft_model2 = AutoModelForSequenceClassification.from_pretrained("./data/peft_model2")
inference_peft_tokenizer2 = AutoTokenizer.from_pretrained("./data/peft_model2")


# Set evaluation mode
task_evaluator = evaluator("text-classification")


# Pretrained Model
eval_results_model = task_evaluator.compute(
    model_or_pipeline=pretrained_model,
    tokenizer=tokenizer,
    data=tokenized_validation_dataset,
    input_column="text",    #default
    label_column="label",   #default
    metric="accuracy",
    label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    strategy="bootstrap",
    # n_resamples=10,
    # random_state=0
)

print(f"""Pretrained Model: {eval_results_model}""")

eval_results_peft_model = task_evaluator.compute(
    model_or_pipeline=inference_peft_model,
    tokenizer=tokenizer,
    data=tokenized_validation_dataset,
    input_column="text",    #default
    label_column="label",   #default
    metric="accuracy",      #default
    label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    strategy="bootstrap",
    # n_resamples=10,
    # random_state=0
)

print(f"""PEFT Model 1: {eval_results_peft_model}""")

eval_results_peft_model2 = task_evaluator.compute(
    model_or_pipeline=inference_peft_model2,
    tokenizer=inference_peft_tokenizer2,
    data=tokenized_validation_dataset,
    input_column="text",    #default
    label_column="label",   #default
    metric="accuracy",
    label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    strategy="bootstrap",
    # n_resamples=10,
    # random_state=0
)

print(f"""PEFT Model 2: {eval_results_peft_model2}""")


Loading adapter weights from ./data/peft_model2 led to unexpected keys not found in the model:  ['pre_classifier.modules_to_save.bias', 'pre_classifier.original_module.bias', 'classifier.modules_to_save.bias', 'classifier.original_module.bias']. 


Pretrained Model: {'accuracy': {'confidence_interval': (0.79, 0.93), 'standard_error': 0.03383853000549755, 'score': 0.87}, 'total_time_in_seconds': 1.2487484359990049, 'samples_per_second': 80.0801803767622, 'latency_in_seconds': 0.012487484359990048}
PEFT Model 1: {'accuracy': {'confidence_interval': (0.8, 0.93), 'standard_error': 0.03291806233629825, 'score': 0.88}, 'total_time_in_seconds': 1.4392801390004024, 'samples_per_second': 69.47917732641764, 'latency_in_seconds': 0.014392801390004026}
PEFT Model 2: {'accuracy': {'confidence_interval': (0.8, 0.93), 'standard_error': 0.03292696204883217, 'score': 0.88}, 'total_time_in_seconds': 1.45092624300014, 'samples_per_second': 68.92149100096631, 'latency_in_seconds': 0.0145092624300014}


In [25]:
# Comparison

def compare_model_outputs(pretrained_model, peft_model_1, peft_model_2):
    # Create a dictionary with the relevant information
    data = {
        "Model": ["Pretrained Model", "PEFT Model 1", "PEFT Model 2"],
        "Accuracy Score": [pretrained_model['accuracy']['score'], peft_model_1['accuracy']['score'], peft_model_2['accuracy']['score']],
        "Confidence Interval Low": [pretrained_model['accuracy']['confidence_interval'][0], peft_model_1['accuracy']['confidence_interval'][0], peft_model_2['accuracy']['confidence_interval'][0]],
        "Confidence Interval High": [pretrained_model['accuracy']['confidence_interval'][1], peft_model_1['accuracy']['confidence_interval'][1], peft_model_2['accuracy']['confidence_interval'][1]],
        "Standard Error": [pretrained_model['accuracy']['standard_error'], peft_model_1['accuracy']['standard_error'], peft_model_2['accuracy']['standard_error']],
        "Total Time (s)": [pretrained_model['total_time_in_seconds'], peft_model_1['total_time_in_seconds'], peft_model_2['total_time_in_seconds']],
        "Samples per Second": [pretrained_model['samples_per_second'], peft_model_1['samples_per_second'], peft_model_2['samples_per_second']],
        "Latency (s)": [pretrained_model['latency_in_seconds'], peft_model_1['latency_in_seconds'], peft_model_2['latency_in_seconds']]
    }

    # Convert the dictionary into a DataFrame
    df = pd.DataFrame(data)

    # Display the DataFrame
    print(df)

compare_model_outputs(eval_results_model, eval_results_peft_model, eval_results_peft_model2)


              Model  Accuracy Score  Confidence Interval Low  \
0  Pretrained Model            0.87                     0.79   
1      PEFT Model 1            0.88                     0.80   
2      PEFT Model 2            0.88                     0.80   

   Confidence Interval High  Standard Error  Total Time (s)  \
0                      0.93        0.033839        1.248748   
1                      0.93        0.032918        1.439280   
2                      0.93        0.032927        1.450926   

   Samples per Second  Latency (s)  
0           80.080180     0.012487  
1           69.479177     0.014393  
2           68.921491     0.014509  


In [28]:
 !zip -r archive.zip data/*

  adding: data/model/ (stored 0%)
  adding: data/model/tokenizer_config.json (deflated 76%)
  adding: data/model/config.json (deflated 46%)
  adding: data/model/model.safetensors (deflated 8%)
  adding: data/model/training_args.bin (deflated 51%)
  adding: data/model/vocab.txt (deflated 53%)
  adding: data/model/tokenizer.json (deflated 71%)
  adding: data/model/special_tokens_map.json (deflated 44%)
  adding: data/peft_model/ (stored 0%)
  adding: data/peft_model/adapter_config.json (deflated 54%)
  adding: data/peft_model/tokenizer_config.json (deflated 76%)
  adding: data/peft_model/README.md (deflated 66%)
  adding: data/peft_model/training_args.bin (deflated 51%)
  adding: data/peft_model/vocab.txt (deflated 53%)
  adding: data/peft_model/adapter_model.safetensors (deflated 7%)
  adding: data/peft_model/tokenizer.json (deflated 71%)
  adding: data/peft_model/special_tokens_map.json (deflated 44%)
  adding: data/peft_model2/ (stored 0%)
  adding: data/peft_model2/adapter_config.jso